In [61]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
df = pd.read_csv('final_dataset.csv')
print(df.head(10))

   Year              Name               College POS  Height (in)  \
0  1987        Mike Adams         Arizona State  CB         69.8   
1  1987      John Adickes                Baylor   C         74.8   
2  1987        Tommy Agee                Auburn  FB         71.8   
3  1987   David Alexander            Tulsa (OK)   C         75.0   
4  1987     Lyneal Alston  Southern Mississippi  WR         72.1   
5  1987      Steve Alvord            Washington  DE         76.5   
6  1987  Anthony Anderson  Grambling State (LA)  CB         73.6   
7  1987    Joe Armentrout             Wisconsin  FB         72.3   
8  1987   Bruce Armstrong            Louisville  OT         76.0   
9  1987       Gene Atkins           Florida A&M  FS         71.3   

   Weight (lbs)  40 Yard  Bench Press  Vert Leap (in)  Broad Jump (in)  \
0           198     4.42           13            32.0              118   
1           266     4.97           25            26.5              103   
2           217     4.81     

In [ ]:
X = df.drop(columns=['Pro Bowl'])
y = df['Pro Bowl']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))